# sqlalchemy 란

- SQLAlchemy는 파이썬에서 사용할 수 있는 ORM(Object-Relational Mapping) 라이브러리로, 데이터베이스와의 상호작용을 추상화하여 데이터베이스를 쉽게 다룰 수 있게 해줍니다. ORM을 통해 데이터베이스 테이블을 파이썬 클래스로 매핑하고, 파이썬 코드를 사용하여 데이터를 쿼리하고 조작할 수 있습니다. 이를 통해 개발자는 SQL 쿼리를 직접 작성하는 대신 객체 지향적인 방식으로 데이터베이스를 다룰 수 있습니다.

- sqlalchemy 구성

![Alt text for broken image link](../resources/sqlalchemy.jpg)

- 데이터베이스와 상호 작용하는 방식은 추상화 수준에 따라  SQL 방식, Core 방식, ORM 방식 3가지로 나뉘어 집니다.

# 데이터베이스 접속
### 1. sqlalchemy 설치

In [ ]:
%pip install SQLAlchemy

### 2. sqlite db 접속 engine 생성

In [ ]:
from sqlalchemy import create_engine

# sqlite:///mydb.db : dialect로 sqlite를 사용하고 db(파일)은 현재 directory의 mydb.db를 사용
# echo=True : 실행되는 SQL을 출력
engine = create_engine('sqlite:///mydb.db', echo=True)

# 데이터베이스와 상호 작용

### 1. SQL 방식
- SQLAlchemy의 가장 기본적인 사용 방식으로, 순수한 SQL 쿼리를 사용하여 데이터베이스와 상호 작용합니다.
- 사용자가 직접 SQL 쿼리를 작성하고 실행할 수 있습니다.
- 데이터베이스와 직접적으로 상호 작용하는 경우나 복잡한 쿼리를 사용해야 하는 경우에 유용합니다.


#### 1-1. 데이터베이스 connection 획득
- engine으로 부터 데이터베이스 connection을 바로 획득합니다.

In [ ]:
conn = engine.connect()

# db file 생성 확인 

#### 1-2. DDL 실행 - Create Table

In [ ]:
from sqlalchemy import text
conn.execute(text("CREATE TABLE TEST (name text PRIMARY KEY, age int, location text)")) 

#### 1-3. Insert 실행

In [ ]:
insert_statement = "INSERT INTO TEST (name, age, location) \
    VALUES (:name, :age, :location)"

test_data = [
    {"name": "Alice", "age": 25, "location":"New York"},
    {"name": "Bob", "age": 30, "location":"Los Angeles"},
    {"name": "Charlie", "age": 35, "location":"Chicago"},
]

conn.execute(text(insert_statement), test_data)
conn.commit()

#### 1-4. Select 실행

In [ ]:
select_statement = "SELECT x, y FROM TEST"
result = conn.execute(text(select_statement))
for row in result:
    print(row)

#### 1-5. DDL 실행 - Drop Table

In [ ]:
conn.execute(text("DROP TABLE TEST")) 

### 2. ORM 방식
- SQLAlchemy의 ORM(Object-Relational Mapping)은 파이썬 클래스와 데이터베이스 테이블을 매핑하여 객체지향 프로그래밍 스타일로 데이터베이스와 상호 작용하는 방법을 제공합니다.
- 데이터베이스 테이블을 파이썬 클래스로 나타내고, 이러한 클래스를 사용하여 데이터를 쿼리하고 조작할 수 있습니다.
- ORM은 데이터베이스와의 상호 작용을 추상화하여 데이터베이스와의 상호 작용을 보다 객체지향적으로 만들어줍니다.
- SQLAlchemy ORM을 사용하면 SQL 쿼리를 직접 작성할 필요 없이 파이썬 객체를 통해 데이터베이스를 조작할 수 있습니다.
#### 2-1. Model기본 Meta정보를 획득
- Dialect로부터 데이터베이스 engine(여기서는 sqlite)에 해당하는 Model(데이터베이스 Table에 매핑되는 Class 객체) 기본 Class를 획득합니다. 

In [ ]:
from sqlalchemy.orm import declarative_base
Base = declarative_base() # Model 기본 Class 획득

#### 2-2. Model 정의
- Model 기본 Class를 상속받아 Model(Table Layout)을 정의합니다.

In [ ]:
"""
prompt :

sqlalchemy.orm.declarative_base 의 결과를 상속 받은 data model 두개를 아래 조건에 맞게 선언하는 code를 작성하라
---
1. 
class 이름 : UserAccount
table 이름 : user_account
칼럼들 :
    nickname (type:string)
    real_name  (type:string)
2. 
class 이름 : Asset
table 이름 : asset
칼럼들 :
    asset_name (type:string)
    owner_id  (type:integer, user_account.id를 참조하는 foreign key)

기타조건 : 
- user_account table 한 row 가 삭제되면 user_asset.owner_id로 연결된 row들도 삭제
- table 객체를 print하면 table의 column과 값을 출력한다.
===
results from ChatGPT-3.5
"""
from sqlalchemy import Column, Integer, String, ForeignKey
from sqlalchemy.orm import relationship

class UserAccount(Base):
    __tablename__ = 'user_account'  # 데이터베이스에서 사용할 테이블 이름입니다.

    id = Column(Integer, primary_key=True)
    nickname = Column(String(30))
    real_name = Column(String)

    # User와 Asset의 관계를 설정합니다.
    assets = relationship("Asset", cascade="all, delete-orphan")

    # 객체를 print 하면 호출됨
    def __repr__(self):
        return f"UserAccount(id={self.id}, nickname={self.nickname}, real_name={self.real_name})"
    
class Asset(Base):
    __tablename__ = 'asset'
    id = Column(Integer, primary_key=True)
    asset_name = Column(String(30))
    
    # User 테이블의 id를 외래키로 설정하여 User와의 관계를 맺습니다.
    owner_id = Column(Integer, ForeignKey('user_account.id'))

    # 객체를 print 하면 호출됨
    def __repr__(self):
        return f"Asset(id={self.id}, asset_name={self.asset_name}, owner_id={self.owner_id})"


- Model 기본 Class(Base)에 Model의 정의된 내용이 등록된 것을 확인합니다.

In [ ]:
print('tables : ', Base.metadata.tables)

#### 2-3. Table들 일괄 Create
- 데이터베이스에 Base 객체에 등록된 Model정보들을 Create Table 명령으로 전환하여 일괄 실행합니다.

In [ ]:
Base.metadata.create_all(engine)

- 전체 Table들 Drop 시에는 아래 code를 실행합니다.

In [ ]:
#Base.metadata.drop_all(engine)

#### 2-4. 데이터베이스 세션(session) 획득
- Session 객체를 통해 Connection Pooling으로부터 데이터베이스 session을 하나 획득합니다.

In [6]:
from sqlalchemy.orm import Session

session = Session(engine)

#### 2-5. Insert 실행

In [7]:
me = UserAccount(nickname='hennry', real_name='김형기')

session.add(me)

In [8]:
session.commit()

2024-04-12 21:51:01,521 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-12 21:51:01,522 INFO sqlalchemy.engine.Engine INSERT INTO user_account (nickname, real_name) VALUES (?, ?)
2024-04-12 21:51:01,522 INFO sqlalchemy.engine.Engine [generated in 0.00076s] ('hennry', '김형기')
2024-04-12 21:51:01,522 INFO sqlalchemy.engine.Engine COMMIT


In [9]:
next_one = UserAccount(nickname='sunsiny', real_name='이순신')

session.add(next_one)

session.rollback()

#### 2-6. Select 실행
- 결과 전체를 Fetch 합니다.

In [10]:
users = session.query(UserAccount).all()

for user in users:
    print(user)

2024-04-12 21:51:29,140 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-12 21:51:29,142 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.nickname AS user_account_nickname, user_account.real_name AS user_account_real_name 
FROM user_account
2024-04-12 21:51:29,142 INFO sqlalchemy.engine.Engine [generated in 0.00092s] ()
UserAccount(id=1, nickname=hennry, real_name=김형기)


- 첫번째 행만 Fetch 합니다.

In [11]:
user = session.query(UserAccount).first()

print(user)

2024-04-12 21:51:42,221 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.nickname AS user_account_nickname, user_account.real_name AS user_account_real_name 
FROM user_account
 LIMIT ? OFFSET ?
2024-04-12 21:51:42,221 INFO sqlalchemy.engine.Engine [generated in 0.00088s] (1, 0)
UserAccount(id=1, nickname=hennry, real_name=김형기)


- 조건에 만족하는 행들만 Fetch 합니다.

In [12]:
users = session.query(UserAccount).filter(UserAccount.nickname=='hennry')

for user in users:
    print(user)

2024-04-12 21:51:55,009 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.nickname AS user_account_nickname, user_account.real_name AS user_account_real_name 
FROM user_account 
WHERE user_account.nickname = ?
2024-04-12 21:51:55,009 INFO sqlalchemy.engine.Engine [generated in 0.00105s] ('hennry',)
UserAccount(id=1, nickname=hennry, real_name=김형기)


#### 2-7. Update 실행

In [13]:
user.real_name = '김형기2'
user.nickname = 'hennry2'
print('Before commit : Is the session dirty?',user in session.dirty)
session.commit()
print('After commit : Is the session dirty?',user in session.dirty)

Before commit : Is the session dirty? True
2024-04-12 21:52:08,018 INFO sqlalchemy.engine.Engine UPDATE user_account SET nickname=?, real_name=? WHERE user_account.id = ?
2024-04-12 21:52:08,018 INFO sqlalchemy.engine.Engine [generated in 0.00104s] ('hennry2', '김형기2', 1)
2024-04-12 21:52:08,018 INFO sqlalchemy.engine.Engine COMMIT
After commit : Is the session dirty? False


#### 2-8. Delete 실행

In [14]:
session.delete(user)
session.commit()

2024-04-12 21:52:30,833 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-12 21:52:30,833 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.nickname AS user_account_nickname, user_account.real_name AS user_account_real_name 
FROM user_account 
WHERE user_account.id = ?
2024-04-12 21:52:30,833 INFO sqlalchemy.engine.Engine [generated in 0.00074s] (1,)
2024-04-12 21:52:30,833 INFO sqlalchemy.engine.Engine SELECT asset.id AS asset_id, asset.asset_name AS asset_asset_name, asset.owner_id AS asset_owner_id 
FROM asset 
WHERE ? = asset.owner_id
2024-04-12 21:52:30,833 INFO sqlalchemy.engine.Engine [generated in 0.00065s] (1,)
2024-04-12 21:52:30,833 INFO sqlalchemy.engine.Engine DELETE FROM user_account WHERE user_account.id = ?
2024-04-12 21:52:30,833 INFO sqlalchemy.engine.Engine [generated in 0.00050s] (1,)
2024-04-12 21:52:30,833 INFO sqlalchemy.engine.Engine COMMIT


#### 2-9. Child Table Insert 실행

In [15]:
me = UserAccount(nickname='tiffanie', real_name='김형기')
me.assets.append(Asset(asset_name="Car"))
me.assets.append(Asset(asset_name="House"))
session.add(me)
session.commit()

2024-04-12 21:52:54,364 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-12 21:52:54,365 INFO sqlalchemy.engine.Engine INSERT INTO user_account (nickname, real_name) VALUES (?, ?)
2024-04-12 21:52:54,365 INFO sqlalchemy.engine.Engine [cached since 112.8s ago] ('tiffanie', '김형기')
2024-04-12 21:52:54,365 INFO sqlalchemy.engine.Engine INSERT INTO asset (asset_name, owner_id) VALUES (?, ?) RETURNING id
2024-04-12 21:52:54,365 INFO sqlalchemy.engine.Engine [generated in 0.00019s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('Car', 1)
2024-04-12 21:52:54,365 INFO sqlalchemy.engine.Engine INSERT INTO asset (asset_name, owner_id) VALUES (?, ?) RETURNING id
2024-04-12 21:52:54,365 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('House', 1)
2024-04-12 21:52:54,365 INFO sqlalchemy.engine.Engine COMMIT


In [16]:
me.assets.append(Asset(asset_name="PC"))
me.assets.append(Asset(asset_name="Phone"))
session.commit()

2024-04-12 21:58:20,153 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-12 21:58:20,154 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.nickname AS user_account_nickname, user_account.real_name AS user_account_real_name 
FROM user_account 
WHERE user_account.id = ?
2024-04-12 21:58:20,154 INFO sqlalchemy.engine.Engine [cached since 349.3s ago] (1,)
2024-04-12 21:58:20,154 INFO sqlalchemy.engine.Engine SELECT asset.id AS asset_id, asset.asset_name AS asset_asset_name, asset.owner_id AS asset_owner_id 
FROM asset 
WHERE ? = asset.owner_id
2024-04-12 21:58:20,154 INFO sqlalchemy.engine.Engine [cached since 349.3s ago] (1,)
2024-04-12 21:58:20,154 INFO sqlalchemy.engine.Engine INSERT INTO asset (asset_name, owner_id) VALUES (?, ?) RETURNING id
2024-04-12 21:58:20,154 INFO sqlalchemy.engine.Engine [cached since 325.8s ago (insertmanyvalues) 1/2 (ordered; batch not supported)] ('PC', 1)
2024-04-12 21:58:20,154 INFO sqlalchemy.engine.Engine INSERT 

In [17]:
for asset in me.assets:
    print(asset)

2024-04-12 21:58:23,232 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-12 21:58:23,233 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.nickname AS user_account_nickname, user_account.real_name AS user_account_real_name 
FROM user_account 
WHERE user_account.id = ?
2024-04-12 21:58:23,233 INFO sqlalchemy.engine.Engine [cached since 352.4s ago] (1,)
2024-04-12 21:58:23,233 INFO sqlalchemy.engine.Engine SELECT asset.id AS asset_id, asset.asset_name AS asset_asset_name, asset.owner_id AS asset_owner_id 
FROM asset 
WHERE ? = asset.owner_id
2024-04-12 21:58:23,233 INFO sqlalchemy.engine.Engine [cached since 352.4s ago] (1,)
Asset(id=1, asset_name=Car, owner_id=1)
Asset(id=2, asset_name=House, owner_id=1)
Asset(id=3, asset_name=PC, owner_id=1)
Asset(id=4, asset_name=Phone, owner_id=1)


#### 2-10. Child Table로부터 Select 실행

- Asset Table을 직접 Select

In [18]:
assets = session.query(Asset).filter(Asset.owner_id==me.id, Asset.asset_name=='Car')

for asset in assets:
    print(asset)

2024-04-12 21:58:50,091 INFO sqlalchemy.engine.Engine SELECT asset.id AS asset_id, asset.asset_name AS asset_asset_name, asset.owner_id AS asset_owner_id 
FROM asset 
WHERE asset.owner_id = ? AND asset.asset_name = ?
2024-04-12 21:58:50,091 INFO sqlalchemy.engine.Engine [generated in 0.00071s] (1, 'Car')
Asset(id=1, asset_name=Car, owner_id=1)


- Table Join 방식으로 Select

In [19]:
rows = session.query(UserAccount, Asset).join(Asset, Asset.owner_id==UserAccount.id)\
    .filter(UserAccount.id==me.id)

for row in rows:
    print(row[0], row[1])

2024-04-12 21:59:45,475 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.nickname AS user_account_nickname, user_account.real_name AS user_account_real_name, asset.id AS asset_id, asset.asset_name AS asset_asset_name, asset.owner_id AS asset_owner_id 
FROM user_account JOIN asset ON asset.owner_id = user_account.id 
WHERE user_account.id = ?
2024-04-12 21:59:45,475 INFO sqlalchemy.engine.Engine [generated in 0.00102s] (1,)
UserAccount(id=1, nickname=tiffanie, real_name=김형기) Asset(id=1, asset_name=Car, owner_id=1)
UserAccount(id=1, nickname=tiffanie, real_name=김형기) Asset(id=2, asset_name=House, owner_id=1)
UserAccount(id=1, nickname=tiffanie, real_name=김형기) Asset(id=3, asset_name=PC, owner_id=1)
UserAccount(id=1, nickname=tiffanie, real_name=김형기) Asset(id=4, asset_name=Phone, owner_id=1)


#### 2-11. Child Table Update 실행

In [ ]:
asset.asset_name = "스포츠 Car"

In [ ]:
session.commit()

#### 2-12. Child Table Delete 실행

In [ ]:
me.assets.remove(asset)

In [ ]:
session.commit()

### 3. Core 방식
- SQLAlchemy Core는 SQL 표현식과 SQL 문을 생성하는 파이썬의 저수준 API입니다.
- 파이썬 코드를 사용하여 SQL 쿼리를 생성하고 실행할 수 있습니다.
- SQL 쿼리를 직접 작성하는 것보다는 추상화된 수준에서 데이터베이스와 상호 작용할 수 있습니다.

#### 3-1. Insert 실행

In [ ]:
from sqlalchemy import insert

stmt = insert(UserAccount).values(nickname='kang', real_name="강인모")

session.execute(stmt)
session.commit()

#### 3-2. Select 실행

In [ ]:
from sqlalchemy import select

stmt = select(UserAccount).where(UserAccount.nickname == 'kang')

cursor = session.execute(stmt)

for row in cursor:
    print(row)

#### 3-3. Update 실행

In [ ]:
from sqlalchemy import update

stmt = update(UserAccount).where(UserAccount.nickname == 'kang').\
    values(real_name='강감찬')

session.execute(stmt)
session.commit()

#### 3-4. Delete 실행

In [ ]:
from sqlalchemy import delete

stmt = delete(UserAccount).where(UserAccount.nickname == 'kang')

session.execute(stmt)
session.commit()

#### 3-5. Child Table Insert 실행

In [ ]:
# Insert User
user_insert = insert(UserAccount).values(nickname='mansour', real_name='만수르')
result = session.execute(user_insert)

user_id = result.lastrowid

# Insert Asset
asset1_insert = insert(Asset).values(asset_name="맨체스터 시티 FC", owner_id=user_id)
asset2_insert = insert(Asset).values(asset_name="돈", owner_id=user_id)

asset1_id = session.execute(asset1_insert).lastrowid
asset2_id = session.execute(asset2_insert).lastrowid

session.commit()

#### 3-6. Select join tables 실행

In [ ]:
stmt = select(UserAccount, Asset).where(UserAccount.id == user_id, UserAccount.id == Asset.owner_id)

rows = session.execute(stmt)

# 결과 출력
for row in rows:
    print("User ID:", row[0].id, "Nickname:", row[0].nickname, "Asset ID:", row[1].id, "Asset Name:", row[1].asset_name)

## Wrap up
1. **SQLAlchemy**: 

    SQLAlchemy는 ORM 라이브러리로, 파이썬 코드를 통해 데이터베이스 테이블을 클래스로 매핑하고, 데이터를 쿼리하고 조작할 수 있습니다.
2. **설치 및 데이터베이스 접속**:

    SQLAlchemy를 설치하고, SQLite 데이터베이스에 연결하는 방법을 배웠습니다.

3. **데이터베이스와의 상호 작용 방법**:
- SQL 방식: 순수 SQL 쿼리를 사용하여 데이터베이스와 상호 작용합니다.
- ORM 방식: 파이썬 클래스와 데이터베이스 테이블을 매핑하여, 객체 지향 프로그래밍 방식으로 데이터베이스와 상호 작용합니다.
- Core 방식: SQLAlchemy Core를 사용하여, 저수준 API를 통해 데이터베이스와 상호 작용합니다.

4. **CRUD 작업**:

    데이터베이스에서 Create, Read, Update, Delete (CRUD) 작업을 수행하는 방법을 ORM 방식과 Core 방식을 통해 배웠습니다.

5. **관계 설정 및 조작**:
 
    사용자(User)와 자산(Asset) 사이의 관계를 설정하고, 관계에 있는 데이터를 조작하는 방법을 배웠습니다. 이 과정에서, 한 사용자가 삭제되면 해당 사용자와 연결된 자산도 함께 삭제되도록 설정하는 방법을 포함합니다.

6. **세션 관리**:

    SQLAlchemy에서 세션을 통해 데이터베이스와의 상호 작용을 관리하는 방법을 배웠습니다. 이를 통해 데이터베이스 작업을 효율적으로 커밋하고, 필요한 경우 롤백할 수 있습니다.